# Image augmentation (luma component)

In [ ]:
import cv2
import os

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
source_path = "/content/drive/MyDrive/Unsharp"

In [ ]:
aug_path = "/content/drive/MyDrive/Augment"
os.mkdir(aug_path)

In [ ]:
for i in os.listdir(source_path):
  os.mkdir(aug_path+'/'+i)

In [ ]:
dest = [aug_path+'/'+i for i in os.listdir(aug_path)]

In [ ]:
dest

['/content/drive/MyDrive/Augment/Mild NPDR(120)',
 '/content/drive/MyDrive/Augment/MODERATE NPDR(120)',
 '/content/drive/MyDrive/Augment/SEVERE NPDR(120)',
 '/content/drive/MyDrive/Augment/Normal(120)',
 '/content/drive/MyDrive/Augment/PDR(120)']

In [ ]:
dest[0].split('/')[5]

'Mild NPDR(120)'

In [ ]:
dest_path = aug_path

In [ ]:
folders = [i.split('/')[5] for i in dest]

In [ ]:
# RGB to YCBCR
def rgb2ycbcr(im_rgb):
    im_rgb = im_rgb.astype(np.float32)
    im_ycrcb = cv2.cvtColor(im_rgb, cv2.COLOR_RGB2YCR_CB)
    im_ycbcr = im_ycrcb[:, :, (0, 2, 1)].astype(np.float32)
    im_ycbcr[:, :, 0] = (im_ycbcr[:, :, 0] * (235 - 16) + 16) / 255.0  # Normalize luma channel
    return im_ycbcr

# Luma component
def extract_luma(ycbcr_image):
    # Split the YCbCr image into separate channels
    y, cb, cr = cv2.split(ycbcr_image)

    # The 'y' channel contains the luma (brightness) information
    return y

In [ ]:
for i in range(len(folders)):
  ip = source_path + '/' + folders[i]
  op = dest_path + '/' + folders[i]
  for j in os.listdir(ip):
    img = cv2.imread(ip+'/'+j)
    ycbcr = rgb2ycbcr(img)
    luma_channel = extract_luma(ycbcr)
    reshaped_img = cv2.resize(luma_channel, (224, 224))
    flipped_image1 = cv2.flip(reshaped_img, 0)
    flipped_image2 = cv2.flip(reshaped_img, 1)
    flipped_image3 = cv2.flip(reshaped_img, -1)

    im2 = np.array(tf.image.random_brightness(reshaped_img, 0.2))
    cv2.imwrite(op +'/' + j, reshaped_img)
    cv2.imwrite(op +'/' + 'bright_'+j, im2)
    cv2.imwrite(op +'/' + 'flipped_1'+j, flipped_image1)
    cv2.imwrite(op +'/' + 'flipped_2'+j, flipped_image2)
    cv2.imwrite(op +'/' + 'flipped_3'+j, flipped_image3)

# Train test split

In [ ]:
src = "/content/drive/MyDrive/Augment"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow import keras
from tensorflow.python.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.optimizers import Adam

In [ ]:
import tensorflow as tf

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
# Training
batch_size=16
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    src,
    validation_split=0.2,
    subset="training",
    seed=123,
    label_mode='categorical',
    image_size=(IMAGE_SIZE),
    batch_size=batch_size
)

# Same for validation
batch_size=16
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    src,
    validation_split=0.2,
    subset="validation",
    seed=123,
    label_mode='categorical',
    image_size=(IMAGE_SIZE),
    batch_size=batch_size
)

Found 3000 files belonging to 5 classes.
Using 2400 files for training.
Found 3000 files belonging to 5 classes.
Using 600 files for validation.


In [ ]:
class_names=train_ds.class_names
print(class_names)

['MODERATE NPDR(120)', 'Mild NPDR(120)', 'Normal(120)', 'PDR(120)', 'SEVERE NPDR(120)']


In [ ]:
!pip install split-folders
import splitfolders
import pathlib

base_ds = src
base_ds = pathlib.Path(base_ds)
img_height=224
img_width=224
batch_size=2
splitfolders.ratio(base_ds, output='images', seed=1321, ratio=(.7,.15,.15), group_prefix=None)

batch_size = 16

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator()

test_ds = datagen.flow_from_directory(
    'images/test',
    target_size = (224, 224),
    batch_size = batch_size,
    class_mode='categorical',
    shuffle=False)

Copying files: 3000 files [00:54, 55.50 files/s] 

Found 450 images belonging to 5 classes.


# Training on Custom NN

In [ ]:
# Filter size for pooling layers = 3 x 3
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

model = keras.Sequential([

                          keras.Input(shape=(224,224,3)),
                          BatchNormalization(),
                          layers.Conv2D(32,3,padding='same', activation='relu', name="First_CNN_Layer"),
                          layers.MaxPooling2D((3,3), name="First_MaxPool_Layer"),
                          # BatchNormalization(),
                          layers.Conv2D(64,3,padding='same', activation='relu',name="Second_CNN_Layer"),
                          layers.MaxPooling2D((3,3),name="Second_MaxPool_Layer"),
                          layers.Conv2D(128,3,padding='same', activation='relu',name="Third_CNN_Layer"),
                          layers.MaxPooling2D((3,3),name="Third_MaxPool_Layer"),
                          # BatchNormalization(),
                          # layers.Conv2D(128,3,padding='same', activation='relu',name="Fourth_CNN_Layer"),
                          # layers.MaxPooling2D((2,2),name="Fourth_MaxPool_Layer"),
                          layers.Flatten(),
                          layers.Dense(128, activation='relu', name="First_Dense_Layer", kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
                          Dropout(0.1,),
                          layers.Dense(64, activation='relu', name="Second_Dense_Layer", kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01)),
                          Dropout(0.1),
                          # BatchNormalization(),
                          layers.Dense(5, activation='softmax', name="Third_Dense_Layer")
])

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_5 (Bat  (None, 224, 224, 3)       12        
 chNormalization)                                                
                                                                 
 module_wrapper_80 (ModuleW  (None, 224, 224, 32)      896       
 rapper)                                                         
                                                                 
 module_wrapper_81 (ModuleW  (None, 74, 74, 32)        0         
 rapper)                                                         
                                                                 
 module_wrapper_82 (ModuleW  (None, 74, 74, 64)        18496     
 rapper)                                                         
                                                                 
 module_wrapper_83 (ModuleW  (None, 24, 24, 64)       

# Model Training

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Fit the model
epochs=40
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5,
                                            min_delta = 1e-5,
                                            verbose = 1,
                                            mode = 'min',
                                            restore_best_weights = True)
history=model.fit(

    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks= [callback]

)

Epoch 1/40
150/150 [==============================] - 12s 63ms/step - loss: 1.3389 - accuracy: 0.4013 - val_loss: 1.2628 - val_accuracy: 0.4200
Epoch 2/40
150/150 [==============================] - 9s 59ms/step - loss: 1.0757 - accuracy: 0.5475 - val_loss: 0.9238 - val_accuracy: 0.6050
Epoch 3/40
150/150 [==============================] - 10s 65ms/step - loss: 0.9247 - accuracy: 0.6154 - val_loss: 0.8450 - val_accuracy: 0.6183
Epoch 4/40
150/150 [==============================] - 10s 64ms/step - loss: 0.7635 - accuracy: 0.6900 - val_loss: 0.8262 - val_accuracy: 0.6550
Epoch 5/40
150/150 [==============================] - 10s 64ms/step - loss: 0.6257 - accuracy: 0.7596 - val_loss: 0.7733 - val_accuracy: 0.6967
Epoch 6/40
150/150 [==============================] - 9s 57ms/step - loss: 0.5162 - accuracy: 0.7954 - val_loss: 0.7460 - val_accuracy: 0.7150
Epoch 7/40
150/150 [==============================] - 10s 64ms/step - loss: 0.4306 - accuracy: 0.8308 - val_loss: 0.8780 - val_accuracy: 0

In [ ]:
score = model.evaluate(test_ds)
print("Loss:",score[0],"Accuracy:",score[1])

29/29 [==============================] - 1s 22ms/step - loss: 0.4640 - accuracy: 0.8422
Loss: 0.4639659523963928 Accuracy: 0.8422222137451172


In [ ]:
model.save_weights("/content/drive/MyDrive/Model_weights/custom/tfmodel")
model.save("/content/drive/MyDrive/Model_weights/custom")

## **4575993 + 3864**

# Resnet

In [ ]:
resnet_model=Sequential()
pretrained_model=tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(224,224,3),
    pooling='avg',
    classes=7)
for layer in pretrained_model.layers:
  layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512,activation='relu'))
resnet_model.add(Dense(256,activation='relu'))
resnet_model.add(Dense(5,activation='softmax'))

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Fit the model
epochs=40
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5,
                                            min_delta = 1e-5,
                                            verbose = 1,
                                            mode = 'min',
                                            restore_best_weights = True)
history=resnet_model.fit(

    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks= [callback]

)

Epoch 1/40
150/150 [==============================] - 38s 190ms/step - loss: 1.0505 - accuracy: 0.5717 - val_loss: 0.8590 - val_accuracy: 0.6300
Epoch 2/40
150/150 [==============================] - 13s 83ms/step - loss: 0.7552 - accuracy: 0.6837 - val_loss: 0.6412 - val_accuracy: 0.7283
Epoch 3/40
150/150 [==============================] - 12s 79ms/step - loss: 0.6404 - accuracy: 0.7404 - val_loss: 0.6709 - val_accuracy: 0.7250
Epoch 4/40
150/150 [==============================] - 12s 81ms/step - loss: 0.5537 - accuracy: 0.7767 - val_loss: 0.5410 - val_accuracy: 0.7800
Epoch 5/40
150/150 [==============================] - 14s 88ms/step - loss: 0.5162 - accuracy: 0.7954 - val_loss: 0.7105 - val_accuracy: 0.7150
Epoch 6/40
150/150 [==============================] - 13s 82ms/step - loss: 0.4198 - accuracy: 0.8275 - val_loss: 0.4709 - val_accuracy: 0.8167
Epoch 7/40
150/150 [==============================] - 12s 79ms/step - loss: 0.3549 - accuracy: 0.8571 - val_loss: 0.6010 - val_accuracy

In [ ]:
score=resnet_model.evaluate(test_ds)
print("Loss:",score[0],"Accuracy:",score[1])

29/29 [==============================] - 4s 87ms/step - loss: 0.1523 - accuracy: 0.9511
Loss: 0.15231762826442719 Accuracy: 0.9511111378669739


In [ ]:
resnet_model.save_weights("/content/drive/MyDrive/Model_weights/resnet/tfmodel")
resnet_model.save("/content/drive/MyDrive/Model_weights/resnet")

# ConvNext

In [ ]:
convnext_model = Sequential()

pretrained_model_3=tf.keras.applications.ConvNeXtTiny(
    model_name="convnext_tiny",
    include_top=False,
    include_preprocessing=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224,224,3),
    pooling='avg',
    classes=5,
    classifier_activation="softmax",
)

for layer in pretrained_model_3.layers:
  layer.trainable=False

convnext_model.add(pretrained_model_3)
convnext_model.add(Flatten())
convnext_model.add(Dense(512, activation= tf.keras.activations.gelu,
                       kernel_regularizer=tf.keras.regularizers.L1(0.03),
                       activity_regularizer=tf.keras.regularizers.L2(0.03))
)
convnext_model.add(tf.keras.layers.Dropout(0.5))
convnext_model.add(Dense(5, activation='softmax'))

111650432/111650432 [==============================] - 1s 0us/step


In [ ]:
convnext_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 convnext_tiny (Functional)  (None, 768)               27820128  
                                                                 
 module_wrapper_4 (ModuleWr  (None, 768)               0         
 apper)                                                          
                                                                 
 module_wrapper_5 (ModuleWr  (None, 512)               393728    
 apper)                                                          
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 module_wrapper_6 (ModuleWr  (None, 5)                 2565      
 apper)                                                          
                                                      

In [ ]:
convnext_model.compile(optimizer=tf.keras.optimizers.AdamW(
    learning_rate=0.0001,
    weight_decay=0.004,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name="AdamW",
),loss='kullback_leibler_divergence', metrics=['accuracy'])

In [ ]:
ep=40
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                            patience=5,
                                            min_delta = 1e-5,
                                            verbose = 1,
                                            mode = 'min',
                                            restore_best_weights = True)
history3 = convnext_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=ep,
    #steps_per_epoch = int(97),
    callbacks=[callback]
)

Epoch 1/40
150/150 [==============================] - 49s 220ms/step - loss: 1.4129 - accuracy: 0.4171 - val_loss: 0.9954 - val_accuracy: 0.5633
Epoch 2/40
150/150 [==============================] - 31s 205ms/step - loss: 1.0173 - accuracy: 0.5671 - val_loss: 0.8599 - val_accuracy: 0.6433
Epoch 3/40
150/150 [==============================] - 27s 176ms/step - loss: 0.8676 - accuracy: 0.6342 - val_loss: 0.7743 - val_accuracy: 0.7033
Epoch 4/40
150/150 [==============================] - 26s 174ms/step - loss: 0.8192 - accuracy: 0.6596 - val_loss: 0.7248 - val_accuracy: 0.7233
Epoch 5/40
150/150 [==============================] - 26s 172ms/step - loss: 0.7590 - accuracy: 0.6829 - val_loss: 0.6842 - val_accuracy: 0.7650
Epoch 6/40
150/150 [==============================] - 26s 173ms/step - loss: 0.7067 - accuracy: 0.7054 - val_loss: 0.6641 - val_accuracy: 0.7650
Epoch 7/40
150/150 [==============================] - 26s 173ms/step - loss: 0.6688 - accuracy: 0.7188 - val_loss: 0.6397 - val_ac

In [ ]:
score=convnext_model.evaluate(test_ds)
print("Loss:",score[0],"Accuracy:",score[1])

29/29 [==============================] - 9s 191ms/step - loss: 0.1800 - accuracy: 0.9467
Loss: 0.1799783557653427 Accuracy: 0.9466666579246521


In [ ]:
convnext_model.save_weights("/content/drive/MyDrive/Model_weights/convnext/tfmodel")
convnext_model.save("/content/drive/MyDrive/Model_weights/convnext")

# Resnet Performance

In [ ]:
from tensorflow import keras

In [ ]:
import os
import cv2
import numpy as np

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Model_weights/resnet')

In [ ]:
class_names = ['MODERATE NPDR(120)', 'Mild NPDR(120)', 'Normal(120)', 'PDR(120)', 'SEVERE NPDR(120)']

In [ ]:
actual, predicted = [], []

In [ ]:
dest_path = "/content/drive/MyDrive/Augment"

IMAGE_SIZE = (224, 224)
for folder in os.listdir(dest_path):
  for img in os.listdir(dest_path+'/'+folder):
    actual.append(class_names.index(folder))
    image = cv2.imread(dest_path+'/'+folder+'/'+img)
    image_resized = cv2.resize(image, (IMAGE_SIZE))
    image = np.expand_dims(image_resized, axis = 0)
    pred = model.predict(image)
    result = class_names[np.argmax(pred)]
    predicted.append(class_names.index(result))

1/1 [==============================] - 0s 27ms/step


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
len(predicted)

3000

In [ ]:
df_dict = {'Actual':actual, 'Predicted':predicted}
df = pd.DataFrame(df_dict)
print(accuracy_score(df['Actual'], df['Predicted']))

0.9476666666666667


In [ ]:
print(confusion_matrix(df['Actual'], df['Predicted']))

[[549  18   3  11  19]
 [ 10 585   4   0   1]
 [  1   6 593   0   0]
 [  0   1   0 587  12]
 [ 25   5   0  41 529]]


In [ ]:
print(classification_report(df['Actual'], df['Predicted']))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93       600
           1       0.95      0.97      0.96       600
           2       0.99      0.99      0.99       600
           3       0.92      0.98      0.95       600
           4       0.94      0.88      0.91       600

    accuracy                           0.95      3000
   macro avg       0.95      0.95      0.95      3000
weighted avg       0.95      0.95      0.95      3000



In [ ]:
df.to_excel('/content/drive/MyDrive/resnet_predictions.xlsx', index=False)

# ConvNext Performance

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Model_weights/convnext')

In [ ]:
actual, predicted = list(), list()

In [ ]:
dest_path = "/content/drive/MyDrive/Augment"

IMAGE_SIZE = (224, 224)
for folder in os.listdir(dest_path):
  for img in os.listdir(dest_path+'/'+folder):
    actual.append(class_names.index(folder))
    image = cv2.imread(dest_path+'/'+folder+'/'+img)
    image_resized = cv2.resize(image, (IMAGE_SIZE))
    image = np.expand_dims(image_resized, axis = 0)
    pred = model.predict(image)
    result = class_names[np.argmax(pred)]
    predicted.append(class_names.index(result))

1/1 [==============================] - 0s 44ms/step


In [ ]:
df_dict = {'Actual':actual, 'Predicted':predicted}
df = pd.DataFrame(df_dict)
print(accuracy_score(df['Actual'], df['Predicted']))

0.9613333333333334


In [ ]:
acc = 0.9613333333333334

In [ ]:
print("Accuracy of ConvNeXt model: ", acc)

Accuracy of ConvNeXt model:  0.9613333333333334


In [ ]:
print(confusion_matrix(df['Actual'], df['Predicted']))

[[567  16   2   0  15]
 [  8 588   2   0   2]
 [  0   7 593   0   0]
 [  2   0   0 584  14]
 [ 17   4   0  27 552]]


In [ ]:
print(classification_report(df['Actual'], df['Predicted']))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95       600
           1       0.96      0.98      0.97       600
           2       0.99      0.99      0.99       600
           3       0.96      0.97      0.96       600
           4       0.95      0.92      0.93       600

    accuracy                           0.96      3000
   macro avg       0.96      0.96      0.96      3000
weighted avg       0.96      0.96      0.96      3000



In [ ]:
df.to_excel('/content/drive/MyDrive/convnext_predictions.xlsx', index=False)

In [ ]:
import keras
import cv2
import os
import numpy as np

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Model_weights/convnext')

In [ ]:
IMAGE_SIZE = (224, 224)
class_names = ['MODERATE NPDR(120)', 'Mild NPDR(120)', 'Normal(120)', 'PDR(120)', 'SEVERE NPDR(120)']

image = cv2.imread("/content/02200281_(0001).jpg")
image_resized = cv2.resize(image, (IMAGE_SIZE))
image = np.expand_dims(image_resized, axis = 0)
pred = model.predict(image)
result = class_names[np.argmax(pred)]
print(result)

1/1 [==============================] - 2s 2s/step
Mild NPDR(120)
